In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **IMPORT MODULE**

In [ ]:
import pandas as pd
import numpy as np
import time
import csv
import os
from tqdm import tqdm

# **READ DATA**

In [ ]:
spotify_id_df = pd.read_csv('/content/drive/MyDrive/Phân tích và trực quan hóa dữ liệu/spotify_id.csv', names=['id'])[1:]

In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 10.7 MB/s eta 0:00:00


# **CRAWL GENRES**

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '39b0ca6f345d479f9ea3258369b7f953'
client_secret = 'd57491f755a14c70858574158e748a0e'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

In [ ]:
def fetch_genres_for_tracks(track_ids):
    artist_genres = {}
    processed_artists = set()
    artist_ids = set()

    # Fetch track details and gather artist IDs
    for i in range(0, len(track_ids), 50):  # Batch of 50 tracks
        batch = track_ids[i:i+50]
        try:
            tracks = sp.tracks(batch)['tracks']
            for track in tracks:
                if track and track.get('artists'):  # Ensure track and artists are valid
                    for artist in track['artists']:
                        if artist['id'] not in processed_artists:
                            artist_ids.add(artist['id'])
        except spotipy.exceptions.SpotifyException as e:
            print(f"Error fetching tracks batch: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")
        time.sleep(5)

    # Fetch genres for artists
    for i in range(0, len(artist_ids), 50):  # Batch of 50 artists
        batch = list(artist_ids)[i:i+50]
        try:
            artists = sp.artists(batch)['artists']
            for artist in artists:
                artist_genres[artist['id']] = artist.get('genres', [])
                processed_artists.add(artist['id'])
        except spotipy.exceptions.SpotifyException as e:
            print(f"Error: {e}")
        time.sleep(5)

    return artist_genres

In [ ]:
def map_genres_to_tracks(track_ids, output_file):
    genres_by_artist = fetch_genres_for_tracks(track_ids)
    for i in range(0, len(track_ids), 50):  # Batch of 50 tracks
        batch = track_ids[i:i+50]
        try:
            tracks = sp.tracks(batch)['tracks']
            for track in tqdm(tracks):
                if track is not None and isinstance(track, dict):
                    track_id = track['id']
                    track_genres = set()
                    for artist in track['artists']:
                        artist_genres = genres_by_artist.get(artist['id'], [])
                        track_genres.update(artist_genres)
                    with open(output_file, mode='a', newline='', encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow([track_id, ';'.join(track_genres)])  # Save genres
                    print(f"Saved: {track_id} -> {list(track_genres)}")
        except spotipy.exceptions.SpotifyException as e:
            print(f"Error: {e}")
        time.sleep(5)


In [ ]:
output_file = '/content/drive/MyDrive/Phân tích và trực quan hóa dữ liệu/output.csv'
if not os.path.exists(output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'genres'])

In [ ]:
track_ids =  list(spotify_id_df['id'].unique()[15000:])
result = map_genres_to_tracks(track_ids, output_file)

 36%|███▌      | 18/50 [00:00<00:00, 174.22it/s]

Saved: 26ObOMushZ72HM9PAEuOOO -> []
Saved: 3hUk7wHp5Ruas0FLLVW69c -> ['tuna', 'portuguese pop']
Saved: 6xMpDmxF3jsTrBBKuYFmJq -> ['norwegian hip hop', 'norwegian pop rap', 'norwegian trap']
Saved: 4fWaQxKUOW3bYJ2R1qDRzR -> []
Saved: 1TNMWejP82ZdXQ7qVszEAo -> ['azontobeats', 'afrobeats', 'nigerian pop', 'viral afropop', 'ghanaian hip hop', 'afropop']
Saved: 7eo9YPXQqZjB1DxYAnvHqy -> ['afrobeats', 'nigerian pop']
Saved: 2noVi0LCf3yUlyARZB1Mw7 -> ['viral afropop']
Saved: 1WhkbEoHWXTeL9dzikqJsY -> ['kazakh hip hop', 'russian underground rap', 'kazakh indie', 'central asian hip hop']
Saved: 6ezYHltHYhuJckdCsYsRJI -> ['k-pop girl group']
Saved: 6SwATVDZUy3jha72jDyxe2 -> ['j-pop']
Saved: 0bB8hmswtN9WvOQXV159CC -> ['trap italiana', 'italian hip hop', 'italian pop']
Saved: 7v4C0Bh0UUIr54Y7h86GPW -> ['italian adult pop', 'rome indie', 'italian pop']
Saved: 0D3QZNAMH2d5MFkVlebo6h -> ['trap italiana', 'italian hip hop', 'italian pop']
Saved: 7EiZI6JVHllARrX9PUvAdX -> ['pop', 'trap', 'atl hip hop',

100%|██████████| 50/50 [00:00<00:00, 174.33it/s]


Saved: 2LCoh1sop5AYzRBVDHFoiK -> ['german hip hop', 'turkish hip hop', 'german drill', 'frauenrap']
Saved: 73JHeIhYKRV1c6RI4ZXsjW -> ['german hip hop']
Saved: 3FkKvUfVnxmxdHrXwuI5vx -> ['german hip hop', 'german pop']
Saved: 4g79mlIRCMtOvi1ZJVpq3t -> ['urbano chileno', 'reggaeton chileno', 'drill chileno']
Saved: 6Y5Bgoz421cN2AioH45NZv -> ['pop dance', 'russian hip hop', 'russian viral rap', 'slap house', 'electro house', 'rap', 'memphis hip hop']
Saved: 3xfiWjgYDcEGMzYTykkk1u -> ['russian hip hop', 'belarusian hip hop', 'russian alt pop']
Saved: 3U7iXmjXJLsgn0EM23q4ag -> ['russian hip hop', 'russian viral rap', 'russian drill', 'russian trap']
Saved: 3eXbmQ58zo1jULMJJzn9jC -> ['russian hip hop', 'jersey club']
Saved: 3P2ubMyuL9NQOVU67TVCXA -> ['funk das antigas', 'funk carioca', 'funk melody', 'funk 150 bpm', 'pop nacional', 'pop teen brasileiro']
Saved: 0fq2RzyyUoqELsONRF8ZzP -> ['german hip hop']
Saved: 4zYHsmc2V9RyR8eOA0mlx6 -> ['trap argentino', 'argentine hip hop', 'trap latino',

 34%|███▍      | 17/50 [00:00<00:00, 166.81it/s]

...

Saved: 6SePnvTZkIXZznc8Ftc6vM -> ['k-pop girl group']
Saved: 0eFRk9wgFPxg6iXhDjvGAo -> ['taiwan pop', 'chinese indie', 'taiwan singer-songwriter']
Saved: 2JusqhuSgWouMe0CXAWcGN -> ['swedish drill', 'swedish trap', 'swedish gangsta rap', 'swedish hip hop', 'swedish trap pop']
Saved: 6Vzl09EVLdB1MTZdDwm2Zd -> ['polish trap', 'polish hip hop']
Saved: 3WX6nwhznaVTD9CazfMV4F -> ['polish trap', 'polish hip hop', 'polish pop']
Saved: 63NqA1azM8ogGsJa7tFCRg -> ['polish viral pop', 'polish pop']
Saved: 0L4BkyT2W2hjR91aVFCauZ -> ['norwegian pop']
Saved: 3r0aM0M0toxEPbkYtXhNAZ -> ['russelater', 'danspunk']
Saved: 4uoSxzvEVMcDsIK0aHZFAv -> ['latin pop', 'dutch house', 'mexican pop', 'dance pop', 'grime', 'latin viral pop', 'pop rap', 'dutch pop']
Saved: 0PGAJhIelmtWlfQVHeFa1A -> ['dutch hip hop']
Saved: 4tvoZzMGhx23HAlS5GUVCa -> ['afrobeats', 'nigerian pop', 'nigerian hip hop', 'afro drill']
Saved: 5qYHD8Mu7HSC9RFzqkLmBA -> ['nasyid']
Saved: 68cUJFq7aAbgeurngTmngL -> ['sad sierreno', 'sierreno', '

 44%|████▎     | 17/39 [00:00<00:00, 162.09it/s]

Saved: 4460AP9uejzHSNqNxTXzIM -> ['icelandic singer-songwriter', 'classic icelandic pop', 'icelandic pop', 'icelandic rock']
Saved: 3Nl5OkkmS5DaBZvuYofpAt -> ['pop']
Saved: 0YbFEoJi7Esa7lhIrEpBjl -> ['modern rock', 'rock', 'pop']
Saved: 0O4jcIw70DqQGHHn0oseUn -> []
Saved: 3LLnO4U1A5CJFtdP7AwasZ -> ['finnish trap', 'finnish hip hop', 'finnish drill']
Saved: 5Om2bI3bo28V7XKvDHYy7d -> ['finnish pop', 'finnish dance pop', 'finnish hip hop', 'finnish drill']
Saved: 6aJc9yW2geBF4OsrqOslOQ -> ['finnish pop', 'finnish dance pop', 'finnish hip hop']
Saved: 2yREzJnpX353biub9EZiAI -> ['estonian hip hop', 'estonian pop']
Saved: 4sGUkfsLxIK4tZxLZt0unU -> ['danish hip hop', 'danish pop']
Saved: 264O5da5Insceb26qjx5tF -> ['danish hip hop', 'danish pop']
Saved: 1dCrghiVlmSKKRXQS95W1T -> ['german hip hop']
Saved: 2Gk6fi0dqt91NKvlzGsmm7 -> ['trap', 'slap house', 'atl hip hop', 'rap', 'hip hop', 'pop rap']
Saved: 5hXCTV8OCsNak8OVry7yzM -> []
Saved: 1DxEbt6OSbrkRthk6pjZue -> ['trap argentino', 'trap trist

100%|██████████| 39/39 [00:00<00:00, 160.44it/s]


Saved: 7E8LK4R9gAkSG4CD1kqvsX -> ['czech hip hop', 'czech drill', 'czsk hip hop', 'czsk emo rap']
Saved: 0SWDToufOxRVIngk8c1oZi -> ['sertanejo', 'sertanejo universitario', 'arrocha']
Saved: 5eID9KNVJV2fDSPQGzQHQg -> ['agronejo', 'sertanejo universitario', 'sertanejo pop', 'arrocha', 'sertanejo']


# **PREPROCESSING**

In [ ]:
output = pd.read_csv('/content/drive/MyDrive/Phân tích và trực quan hóa dữ liệu/output.csv')

In [ ]:
# check duplicate
unique_rows = []
unique = set()
for index, row in output.iterrows():
  row_tuple = tuple(row)
  if row_tuple not in unique:
    unique_rows.append(row)
    unique.add(row_tuple)

In [ ]:
output_unique = pd.DataFrame(unique_rows, columns=output.columns)
output_unique.head(10)

,id,genres
0,2plbrEY59IikOBgBGLjaoe,art pop;dance pop;pop
1,5vNRhkKd0yEAg8suGBpjeY,dance pop;pop;k-pop
2,6dOtVTDdiauQNBQEDOtlAB,art pop;pop
3,2CGNAOSuO1MEFCbBRgUzjd,pop;west coast rap;r&b;conscious hip hop;rap;h...
4,06qMRF18gwbOYYbnP2du6i,soft rock;new romantic;new wave;new wave pop
5,0bYg9bo50gSsH3LtXe2SQn,dance pop;pop;urban contemporary
6,7ne4VBA60CxGM75vw0EYad,alt z
7,4lriIG2vNqwDWzOj2I9rtj,french pop;belgian pop;nouvelle chanson franca...
8,7tI8dRuH2Yc6RuoTjxo4dU,k-pop
9,2qSkIjg1o9h3YT9RAgYN75,pop


In [ ]:
len(spotify_id_df['id'].unique())

15439

In [ ]:
output_unique.shape

(15438, 2)

In [ ]:
output_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15438 entries, 0 to 15639
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      15438 non-null  object
 1   genres  13496 non-null  object
dtypes: object(2)
memory usage: 361.8+ KB


In [ ]:
output_unique.to_csv('/content/drive/MyDrive/Phân tích và trực quan hóa dữ liệu/genres.csv', index=False)